In [ ]:
# 1. Install Dependencies and Setup
%pip install tensorflow tensorflow-gpu opencv-python matplotlib
%pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')


In [ ]:
# 2. Remove dodgy images
import cv2
import imghdr

data_dir = 'ISIC/train'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']
allowed_classes =['basal cell carcinoma', 'melanoma', 'squamous cell carcinoma']

for image_class in os.listdir(data_dir):
    if image_class not in allowed_classes:
        continue 
    
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom

# Load Data with Data Augmentation
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),       # Randomly flip images horizontally
    RandomRotation(0.2),            # Randomly rotate images by a maximum of 20%
    RandomZoom(0.2)                 # Randomly zoom images by a maximum of 20%
])

data = tf.keras.utils.image_dataset_from_directory(
    'ISIC/train', 
    labels='inferred', 
    label_mode='int', 
    validation_split=0.2,           # Use 20% of the data for validation
    subset='training',              # Specify training subset for data augmentation
    seed=123,                       # Set a random seed for reproducibility
    image_size=(256, 256),          # Set the desired image size for data augmentation
    batch_size=32                   # Set the batch size
)

# Increase dataset size using data augmentation
def generator():
    for batch in data:
        # Convert the original images to float32 before applying data augmentation
        original_images = tf.image.convert_image_dtype(batch[0], tf.float32)
        for i in range(5):  # Augment each batch 5 times
            augmented_images = data_augmentation(original_images, training=True)  # Apply data augmentation
            # Keep the original labels for the augmented images
            yield augmented_images, batch[1]

augmented_data = tf.data.Dataset.from_generator(generator, output_signature=(tf.TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32), tf.TensorSpec(shape=(None,), dtype=tf.int32)))

print("New size of the dataset after data augmentation:", len(list(augmented_data.as_numpy_iterator())))

data_iterator = augmented_data.as_numpy_iterator()
batch = data_iterator.next()
fig, ax = plt.subplots(nrows=5, ncols=4, figsize=(20, 20))
for row in range(5):
    for col in range(4):
        idx = row * 4 + col
        img = batch[0][idx]
        label = batch[1][idx]
        ax[row, col].imshow(img)
        ax[row, col].title.set_text(label)
plt.show()

In [ ]:
# 4. Scale Data
data = data.map(lambda x, y: (x / 255, y))
data.as_numpy_iterator().next()


In [ ]:
# 5. Split Data
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = int(len(data) * 0.1)
train_size, val_size, test_size

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)


In [ ]:
# 6. Build Deep Learning Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 4 output classes
model.compile('adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

In [ ]:
# 7. Train
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=30, validation_data=val, callbacks=[tensorboard_callback])


In [ ]:
# 8. Plot Performance
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# 9. Evaluate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
re = Recall()
acc = BinaryAccuracy()
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    yhat_classes = np.argmax(yhat, axis=-1)  # Get the predicted class indices
    
    pre.update_state(y, yhat_classes)
    re.update_state(y, yhat_classes)
    acc.update_state(y, yhat_classes)
print(pre.result(), re.result(), acc.result())


In [ ]:
# 10. Test
import cv2

In [ ]:

img = cv2.imread('ISIC/Train/melanoma/ISIC_0000013.jpg')
plt.imshow(img)
plt.show()

resize = tf.image.resize(img, (256, 256))
plt.imshow(resize.numpy().astype(int))
plt.show()

yhat = model.predict(np.expand_dims(resize / 255, 0))
predicted_class = np.argmax(yhat)  # Get the index of the highest probability class

classes = ['basal cell carcinoma', 'melanoma', 'squamous cell carcinoma']
print(f'score : {[predicted_class]}')
if classes[predicted_class] in allowed_classes:
    print(f'score : {[yhat]}')
    print(f'Predicted class is a {classes[predicted_class]}')
else:
    print('Error: Image does not belong to skin cancer class.')

In [ ]:
# 11. Save the Model
from tensorflow.keras.models import load_model

model.save(os.path.join('models', 'imageclassifiers.h5'))
new_model = load_model('models/imageclassifiers.h5')
new_model.predict(np.expand_dims(resize / 255, 0))
